# Lexical search of Hyginus, *Fabulae*

> *Add link here to all the caveats...*


### How to use this notebook

1. First, run step 1 (e.g., by selecting the cell labelled **Step 1: load everything** and choosing "Run all below" from the "Cell" menu).  This will be slow, and your mileage may vary depending on how well your connection to different resources on the internet happens to be performing just then.
2. Just below the cell labelled **Step 2: search by example**, fill in between quotation marks an argument to the function `moreLikeThis`.  E.g., to find all forms of `duco` in Hyginus, you could edit that cell to read

    `moreLikeThis("duxit")`

Then run the cell (e.g., by selecting it, and choosing "Run cells" from the "Cell" menu).

# Step 2: search by example

In [27]:
moreLikeThis("A_WORD_OCCURING_IN_TEXT_OF_HYGINUS")

'A_WORD_OCCURING_IN_TEXT_OF_HYGINUS' does not occur in Hyginus.  Check the form again.


# Step 1: load everything


In [1]:
// set up notebook to find repository
val personalRepo = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(personalRepo)

personalRepo: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
// ivy imports
import $ivy.`edu.holycross.shot::latincorpus:7.0.0-pr3`

Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/latincorpus_2.12/7.0.0-preview/latincorpus_2.12-7.0.0-preview.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/latincorpus_2.12/7.0.0-preview/latincorpus_2.12-7.0.0-preview.pom.sha1
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/latincorpus_2.12/7.0.0-preview/latincorpus_2.12-7.0.0-preview.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/seqcomp_2.12/2.2.1/seqcomp_2.12-2.2.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/tabulae_2.12/7.0.5/tabulae_2.12-7.0.5.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/histoutils_2.12/2.3.0/histoutils_2.12-2.3.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/mid/orthography_2.12/2.1.0/orthography_2.12-2.1.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/latphone_2.12/3.0.0/latphone_2.12-3.0.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/seqcomp_2.

import $ivy.$                                              

In [4]:
import edu.holycross.shot.latincorpus._

import edu.holycross.shot.latincorpus._

hyginusUrl: String = "https://raw.githubusercontent.com/LinguaLatina/analysis/master/data/hyginus/hyginus-latc.cex"
hyginus: LatinCorpus = LatinCorpus(
  Vector(
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.0"),
        "EXCERPTA"
      ),
      LexicalToken,
      Vector()
    ),
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.1"),
        "EX"
      ),
      LexicalToken,
      Vector(IndeclinableForm("ls.n16519", "", "", Conjunction))
    ),
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.2"),
        "HYGINI"
      ),
      LexicalToken,
      Vector()
    ),
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.3"),
        "GENEALOGIIS"
      ),
      LexicalToken,
      Vector()
    ),
    LatinParsedToken(
      CitableNode(

In [5]:
val hyginusUrl = "https://raw.githubusercontent.com/LinguaLatina/analysis/master/data/hyginus/hyginus-latc.cex"
val hyginus = LatinCorpus.fromUrl(hyginusUrl)

hyginusUrl: String = "https://raw.githubusercontent.com/LinguaLatina/analysis/master/data/hyginus/hyginus-latc.cex"
hyginus: LatinCorpus = LatinCorpus(
  Vector(
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.0"),
        "EXCERPTA"
      ),
      LexicalToken,
      Vector()
    ),
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.1"),
        "EX"
      ),
      LexicalToken,
      Vector(IndeclinableForm("ls.n16519", "", "", Conjunction))
    ),
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.2"),
        "HYGINI"
      ),
      LexicalToken,
      Vector()
    ),
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.3"),
        "GENEALOGIIS"
      ),
      LexicalToken,
      Vector()
    ),
    LatinParsedToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa126

In [28]:
def moreLikeThis(s: String) = {
    try {
        val lexemes = hyginus.tokenLexemeIndex(s)     
        val labelledLexemes = hyginus.tokenLabelledLexemeIndex(s)  
        println(s"${s} matched ${labelledLexemes.size} lexeme(s): " + labelledLexemes.mkString(", ") + "\n") 
        
        for (lex <- lexemes) {
            val cns = hyginus.citableUnits.sequences.filter( cn => cn.matchesLexeme(lex))
            println("For " + lex + ", matched " + cns.size + " citable passages.\n")
            for ((cn, i) <- cns.zipWithIndex) {
                val txt = cn.lexicalText.mkString(" ")
                println(s"${i + 1} ${txt}\n\n")
            }
        }
        
        
    } catch {
        case nope: java.util.NoSuchElementException => {
            println("'" + s + "' does not occur in Hyginus.  Please check the form, and try again.")
        }    
        
        
    }

}

defined function moreLikeThis